In [1]:
import pandas as pd
import sqlite3 as sql
import ipywidgets as widgets
from ipyleaflet import Map, Polyline, Marker, Popup, basemaps, LayersControl
from IPython.display import display

# Connect to the SQLite database
conn = sql.connect('flights.db')
cur = conn.cursor()

# Select the required columns from the airport_data table
query = "SELECT Airport, lat_decimal, lon_decimal FROM airport_data;"
cur.execute(query)
all_destinations_data = cur.fetchall()

# Close the connection to the database
conn.close()

# Convert the results into a DataFrame
flights_df = pd.DataFrame(all_destinations_data, columns=['Airport', 'lat_decimal', 'lon_decimal'])

# Coordinates for Dublin
dublin_coords = (53.4268, -6.2562)

# Initialize the map centered at Dublin
m = Map(center=dublin_coords, zoom=6, basemap=basemaps.OpenStreetMap.Mapnik)

# Function to update the map based on the selected airports
def update_map(selected_airports):
    m.layers = [m.layers[0]]  # Reset to only keep the base layer

    # Add marker for Dublin
    dublin_marker = Marker(location=dublin_coords)
    m.add_layer(dublin_marker)

    # Add flight paths with popups
    for airport in selected_airports:
        airport_data = flights_df[flights_df['Airport'] == airport]
        for _, row in airport_data.iterrows():
            path = [(dublin_coords[0], dublin_coords[1]), (row['lat_decimal'], row['lon_decimal'])]
            polyline = Polyline(locations=path, color="blue", fill=False)
            popup = Popup(
                location=(row['lat_decimal'], row['lon_decimal']),
                child=widgets.HTML(value=f"<b>{airport}</b>"),
                close_button=False,
            )
            polyline.popup = popup
            m.add_layer(polyline)

# Create a slider widget for selecting the number of airports
num_airports_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=len(flights_df),
    step=1,
    description='Number of Airports:',
    continuous_update=False
)

# Create a dropdown menu for selecting specific airports
airport_select = widgets.SelectMultiple(
    options=flights_df['Airport'].unique().tolist(),
    value=[flights_df['Airport'].unique()[0]],
    description='Airports:',
    disabled=False
)

# Link the slider and the dropdown menu to the update function
def on_slider_change(change):
    selected_airports = flights_df['Airport'].unique()[:change['new']]
    update_map(selected_airports)

def on_airport_change(change):
    update_map(change['new'])

num_airports_slider.observe(on_slider_change, names='value')
airport_select.observe(on_airport_change, names='value')

# Display the map, the slider, and the dropdown menu
display(m)
display(num_airports_slider)
display(airport_select)


Map(center=[53.4268, -6.2562], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

IntSlider(value=1, continuous_update=False, description='Number of Airports:', max=90, min=1)

SelectMultiple(description='Airports:', index=(0,), options=('KEFLAVIK', 'TORONTO/LESTER B. PEARSON INTL, ON',…